In [14]:
# imports
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

import joblib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

PATH = os.getcwd()
VERSION = "Sv6"
FILE = "metadatabase_surrogate_"+VERSION+".csv"
seed = 30
# consolidated pipelines
PROBLEM_FILE = ''
# PROBLEM_FILE = "validation_biagio_benchmark.csv"
features_4bench = ['attr_conc.mean','attr_conc.sd','attr_ent.mean',
    'attr_ent.sd','attr_to_inst','cohesiveness.mean','cohesiveness.sd',
    #'cor.mean',#'cor.sd',
    'cov.mean',#'cov.sd',
    'eigenvalues.mean','eigenvalues.sd',
    'inst_to_attr','iq_range.mean','iq_range.sd',
    #'kurtosis.mean','kurtosis.sd',
    'mad.mean','mad.sd',
    #'max.mean','max.sd','mean.mean','mean.sd',
    'median.mean',
    'median.sd',#'min.mean','min.sd',
    'nr_attr','nr_cor_attr','nr_inst','one_itemset.mean',
    'one_itemset.sd',
    #'range.mean','range.sd',
    'sd.mean','sd.sd'
    #,'skewness.mean', 'skewness.sd'
    ,'sparsity.mean','sparsity.sd','t2','t3','t4','t_mean.mean',
    't_mean.sd','two_itemset.mean','two_itemset.sd','var.mean','var.sd',
    'wg_dist.mean','wg_dist.sd',
    'sil', 'dbs', 'ari'
]

In [15]:
# sv6 light
features_4bench = ['attr_ent.sd','sparsity.sd', 'cov.mean','var.mean','eigenvalues.mean','sparsity.mean', 'wg_dist.sd', 'iq_range.mean','sil','dbs','ari']

In [13]:
len(features_4bench)

11

In [22]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, explained_variance_score, root_mean_squared_error
import numpy as np

# Load dataset
df_surrogate = pd.read_csv("Surrogate_MetaDataset_sv6.csv")
data = df_surrogate[features_4bench]

# Prepare features and target
x_train, y_train = data.values[:, :-1], data.values[:, -1]

# Initialize XGBoost regressor
xgb_regressor = xgb.XGBRegressor(random_state=seed, n_estimators=100, n_jobs=-1)

# Custom scoring functions
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
r2_scorer = make_scorer(r2_score)
explained_variance_scorer = make_scorer(explained_variance_score)

# Perform cross-validation for each metric
rmse_scores = cross_val_score(xgb_regressor, x_train, y_train, cv=5, scoring=rmse_scorer)
mae_scores = cross_val_score(xgb_regressor, x_train, y_train, cv=5, scoring=mae_scorer)
r2_scores = cross_val_score(xgb_regressor, x_train, y_train, cv=5, scoring=r2_scorer)
explained_variance_scores = cross_val_score(xgb_regressor, x_train, y_train, cv=5, scoring=explained_variance_scorer)

# Convert negative MSE scores to positive RMSE scores
rmse_scores = -rmse_scores
mae_scores = -mae_scores  # MAE scorer returns negative values, so we negate them

# Output the results
print("Average RMSE: ", rmse_scores.mean())
print("Average MAE: ", mae_scores.mean())
print("Average R²: ", r2_scores.mean())
print("Average Explained Variance: ", explained_variance_scores.mean())


Average RMSE:  0.07875472819149733
Average MAE:  0.057898535876126464
Average R²:  0.8453714158903454
Average Explained Variance:  0.8454482685395114


In [11]:
data.ari.max()

1.0

In [8]:
xgb_regressor = xgb.XGBRegressor(random_state=seed, n_estimators=100, n_jobs=-1)
xgb_regressor.fit(x_train, y_train)

# rf_regressor = RandomForestRegressor(random_state=seed, n_estimators=100, n_jobs=-1)
# rf_regressor.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=-1,
             num_parallel_tree=None, random_state=30, ...)

In [9]:
joblib.dump(xgb_regressor, f"models/sv6_light.joblib", compress=5)

['models/sv6_light.joblib']

---

In [ ]:
seed = 30

df_surrogate = pd.read_csv("Surrogate_MetaDataset_sv6.csv")
data = df_surrogate[['sil', 'dbs', 'ari']]

x_train, y_train = data.values[:, :-1], data.values[:, -1]
rf_regressor = RandomForestRegressor(random_state=seed, n_estimators=100, n_jobs=-1)
rf_regressor.fit(x_train, y_train)
joblib.dump(rf_regressor, f"models/cvi_sv6.joblib", compress=5)

In [ ]:
# validation
consolidated_pipelines = pd.read_csv('consolidated_pipelines.csv')
biagio_pipelines = consolidated_pipelines[consolidated_pipelines.Tags == '["0_Biagio"]'].reset_index(drop=True)
# X_test = biagio_pipelines + metafeatures per dataset
validation_df = pd.read_csv("validation_biagio_benchmark.csv")
mfs = validation_df.drop_duplicates(subset='file_name', keep='first')
mfs = mfs.drop(['algorithm','sil','dbs','ari','cluster_diff','predicted_cluster','clusters'], axis=1).reset_index(drop=True)
mfs = mfs.rename(columns={'file_name':'dataset'})
biagio_pipelines_ = biagio_pipelines.copy()
biagio_pipelines_ = biagio_pipelines_.merge(mfs, on='dataset')
biagio_pipelines_ = biagio_pipelines_.drop(['filename','surrogate_score','pipeline'], axis=1)
test_data = biagio_pipelines_[features_4bench]
X_test, y_test = test_data.values[:, :-1], test_data.values[:, -1]


In [ ]:
for seed in range(100):
    rf_regressor = RandomForestRegressor(random_state=seed, n_estimators=100, n_jobs=-1)
    rf_regressor.fit(x_train, y_train)
    # Make predictions on the test set
    
    y_pred = rf_regressor.predict(X_test)
    validation_data = biagio_pipelines_.copy()
    validation_data['surrogate_score'] = y_pred
    validation_data_ = validation_data.copy()
    max_scores = validation_data_.loc[validation_data_.groupby('dataset')['surrogate_score'].transform(max) == validation_data_['surrogate_score']]
    print(f"Seed: {seed}")
    print(f"Mean: {round(max_scores.groupby('dataset').agg({'ari':['mean','median','std','count']})['ari']['mean'].mean(),4)}")


In [ ]:
# Create an XGBoost regressor
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Train the model
xgb_regressor.fit(x_train, y_train)

In [ ]:
# validation
consolidated_pipelines = pd.read_csv('consolidated_pipelines.csv')
biagio_pipelines = consolidated_pipelines[consolidated_pipelines.Tags == '["0_Biagio"]'].reset_index(drop=True)

In [ ]:
# X_test = biagio_pipelines + metafeatures per dataset
validation_df = pd.read_csv("validation_biagio_benchmark.csv")
mfs = validation_df.drop_duplicates(subset='file_name', keep='first')
mfs = mfs.drop(['algorithm','sil','dbs','ari','cluster_diff','predicted_cluster','clusters'], axis=1).reset_index(drop=True)
mfs = mfs.rename(columns={'file_name':'dataset'})


In [ ]:
biagio_pipelines_ = biagio_pipelines.copy()
biagio_pipelines_ = biagio_pipelines_.merge(mfs, on='dataset')
biagio_pipelines_ = biagio_pipelines_.drop(['filename','surrogate_score','pipeline'], axis=1)
biagio_pipelines_

In [ ]:
# Make predictions on the test set
test_data = biagio_pipelines_[features_4bench]
X_test, y_test = test_data.values[:, :-1], test_data.values[:, -1]

y_pred = rf_regressor.predict(X_test)

In [ ]:
# Make predictions on the test set
test_data = biagio_pipelines_[features_4bench]
X_test, y_test = test_data.values[:, :-1], test_data.values[:, -1]

y_pred = xgb_regressor.predict(X_test)

In [ ]:
# insert surrogate scores in the df
# mudei o SURROGATE SCORE -> agora ele maximiza
validation_data = biagio_pipelines_.copy()
validation_data['surrogate_score'] = y_pred
validation_data

---

In [ ]:
validation_data_ = validation_data.copy()

validation_data_['Max Surrogate Score'] = validation_data_.groupby('dataset')['surrogate_score'].transform('max')

min_score_df = validation_data_[validation_data_['surrogate_score'] == validation_data_['Max Surrogate Score']]
min_score_df = min_score_df.drop(columns='Max Surrogate Score')
min_score_df.ari.mean()

In [ ]:
validation_data_ = validation_data.copy()
max_scores = validation_data_.loc[validation_data_.groupby('dataset')['surrogate_score'].transform(max) == validation_data_['surrogate_score']]
max_scores.groupby("dataset").agg({'ari':['mean','median','std','count']})['ari']['mean'].mean()

In [ ]:
joblib.dump(rf_regressor, f"models/sv6.joblib")